ICR - Identify Age_Related Conditions
=====================================

[Kaggle Competition](https://www.kaggle.com/competitions/icr-identify-age-related-conditions)

icr_notebook_290623
Based on: [gusthema/identifying-age-related-conditions-w-tfdf](https://www.kaggle.com/code/gusthema/identifying-age-related-conditions-w-tfdf)

In [7]:
# Install packages if they not installed already, otherwise don;t waste time
import pkg_resources

def install(package):
    try:
        pkg_resources.get_distribution(package)
        print("{} ({}) is already installed".format(package, pkg_resources.get_distribution(package).version))
    except pkg_resources.DistributionNotFound:
        print("{} is NOT installed. Installing it.".format(package))
        %pip install {package}

install('scikit-learn')
install('tensorflow')
install('tensorflow_decision_forests')

scikit-learn (1.2.2) is already installed
tensorflow (2.12.0) is already installed
tensorflow_decision_forests (1.3.0) is already installed


# Data Exploration

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# read in data
input_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

# df = pd.read_csv('../data/train.csv')
input_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [32]:
import math

# All stats placeholder
stats_df = pd.DataFrame(columns=["Column", "Min", "1st Q", "Median", "Mean", "3rd Q", "Max"])

# get the features columns
features_cols = input_df.select_dtypes(include=[np.number]).columns[1:-1]

# create a subplot grid
cols = 1
rows = math.ceil(len(features_cols) / cols)
fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))  # adjust the size as needed

# flatten axes for easy iteration
axes = axes.flatten()

for i, col in enumerate(features_cols):
    if input_df[col].isna().any():
        input_sr = input_df[col].dropna()
        col_name = f"{col} (NaN dropped)"
    else:
        input_sr = input_df[col]
        col_name = f"{col}"

    min = input_sr.min()
    mean = input_sr.mean()
    median = input_sr.median()
    q1 = input_sr.quantile(0.25)
    q3 = input_sr.quantile(0.75)
    max = input_sr.max()

    stats_df.loc[i] = [col_name, min, q1, median, mean, q3, max]

    # Plot with histplot and kde
    ax = axes[i]
    sns.histplot(input_sr, kde=True, ax=ax)

    # Replace all `plt.` with `ax.`
    ax.text(0.5, 0.9, col_name, size=40, ha='center', va='center', transform=ax.transAxes)

    ax.axvline(mean, color='red', linestyle='--')
    ax.axvline(median, color='green', linestyle='--')
    ax.axvline(q1, color='yellow', linestyle='--')
    ax.axvline(q3, color='yellow', linestyle='--')

    # Annotations
    y = ax.get_ylim()[1]
    offset = y * 0.13  # 5% of the y-axis range
    ax.text(mean, y-offset, 'Mean', rotation=90, color='red')
    ax.text(median, y-offset, 'Median', rotation=90, color='green')
    ax.text(q1, y-offset, '1st Q', rotation=90, color='yellow')
    ax.text(q3, y-offset, '3rd Q', rotation=90, color='yellow')
    
    # Add horizontal line at max count and label it
    hist, bins = np.histogram(input_sr, bins='auto')
    max_count = hist.max()
    ax.axhline(max_count, color='blue', linestyle='-')
    ax.text(ax.get_xlim()[1]/2, max_count - offset, f'Max Count: {max_count}', ha='center', color='blue')

    # Show table with stats inside the plot
    stats = f'''
            Min: {min:.2f}
            1st Q: {q2:.2f}
            Median: {median:.2f}
            Mean: {mean:.2f}
            3rd Q: {q3:.2f}
            Max: {max:.2f}
    '''
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.75, 0.95, stats, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)

# remove unused subplots
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

print(stats_df.to_string())

plt.tight_layout()
plt.show()

              Column          Min        1st Q       Median          Mean         3rd Q            Max
0                 AF   192.593280  2197.345480  3120.318960   3502.013221   4361.637390   28688.187660
1                 AH    85.200147    85.200147    85.200147    118.624513    113.739540    1910.123198
2                 AM     3.177522    12.270314    20.533110     38.968552     39.139886     630.518230
3                 AR     8.138688     8.138688     8.138688     10.128242      8.138688     178.943634
4                 AX     0.699861     4.128294     5.031912      5.545576      6.431634      38.270880
5                 AY     0.025578     0.025578     0.025578      0.060320      0.036845      10.315851
6                 AZ     3.396778     8.129580    10.461320     10.566447     12.969516      38.971568
7                 BC     1.229900     1.229900     1.229900      8.053012      5.081244    1463.693448
8                BD   1693.624320  4155.702870  4997.960730   5350.388655

# Data Cleaning

# Model

## Data splitting

In [3]:
from sklearn.model_selection import KFold
import tensorflow as tf
import tensorflow_decision_forests as tfdf

2023-06-30 19:43:07.693292: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-30 19:43:07.698378: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-30 19:43:07.848315: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-30 19:43:07.849829: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 19:43:09.669745: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
# Creates a GroupKFold with 5 splits
kf = KFold(n_splits=5, shuffle=True)

# Create list of ids for the creation of oof dataframe.
ID_LIST = input_df.index

# Create a dataframe of required size with zero values.
oof = pd.DataFrame(data=np.zeros((len(ID_LIST),1)), index=ID_LIST)

# Create an empty dictionary to store the models trained for each fold.
models = {}

# Create empty dict to save metrics for the models trained for each fold.
accuracy = {}
cross_entropy = {}

# Save the name of the label column to a variable.
label = "Class"

# Calculate the number of samples for each label.
neg, pos = np.bincount(input_df['Class'])

# Calculate total samples.
total = neg + pos

# Calculate the weight for each label.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

FEATURE_COLUMNS = [i for i in input_df.columns if i not in ["Id"]]

## Model training

In [5]:
# Loop through each fold
for i, (train_index, valid_index) in enumerate(kf.split(X=input_df)):
    print('##### Fold',i+1)

    # Fetch values corresponding to the index 
    train_df = input_df.iloc[train_index]
    valid_df = input_df.iloc[valid_index]
    valid_ids = valid_df.index.values
    
    # Select only feature columns for training.
    train_df = train_df[FEATURE_COLUMNS]
    valid_df = valid_df[FEATURE_COLUMNS]
    
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label=label)
    valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df, label=label)

    # Define the model and metrics
    rf = tfdf.keras.RandomForestModel(
        num_trees=500,   # Number of trees to use in the ensemble
        max_depth=8,     # Max depth of the trees
        min_examples=5,  # Min samples per leaf
    )
    rf.compile(metrics=["accuracy", "binary_crossentropy"]) 
    
    # Model Training
    rf.fit(x=train_ds, class_weight=class_weight)
    models[f"fold_{i+1}"] = rf
    
    
    # Predict OOF value for validation data
    predict = rf.predict(x=valid_ds)
    
    # Store the predictions in oof dataframe
    oof.loc[valid_ids, 0] = predict.flatten() 
    
    # Evaluate and store the metrics in respective dicts
    evaluation = rf.evaluate(x=valid_ds,return_dict=True)
    accuracy[f"fold_{i+1}"] = evaluation["accuracy"]
    cross_entropy[f"fold_{i+1}"]= evaluation["binary_crossentropy"]

##### Fold 1


Use /tmp/tmpqs5xaerq as temporary training directory
Reading training dataset...


2023-06-30 19:43:12.239880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_55' with dtype double and shape [493]
	 [[{{node Placeholder/_55}}]]


Training dataset read in 0:00:08.266685. Found 493 examples.
Training model...
Model trained in 0:00:00.389008
Compiling model...


[INFO 23-06-30 19:43:20.6416 BST kernel.cc:1242] Loading model from path /tmp/tmpqs5xaerq/model/ with prefix c72347e15ba64c29
[INFO 23-06-30 19:43:20.7019 BST decision_forest.cc:660] Model loaded with 500 root(s), 21280 node(s), and 56 input feature(s).
[INFO 23-06-30 19:43:20.7020 BST abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-06-30 19:43:20.7020 BST kernel.cc:1074] Use fast generic engine
2023-06-30 19:43:20.802083: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [493]
	 [[{{node Placeholder/_5}}]]


...

[INFO 23-06-30 19:43:38.9696 BST kernel.cc:1242] Loading model from path /tmp/tmpa3c6nl0c/model/ with prefix 5189018ebb4e425d
[INFO 23-06-30 19:43:39.0285 BST decision_forest.cc:660] Model loaded with 500 root(s), 22038 node(s), and 56 input feature(s).
[INFO 23-06-30 19:43:39.0285 BST kernel.cc:1074] Use fast generic engine
2023-06-30 19:43:39.055265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_24' with dtype double and shape [494]
	 [[{{node Placeholder/_24}}]]


Model compiled.


2023-06-30 19:43:39.781255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_46' with dtype double and shape [123]
	 [[{{node Placeholder/_46}}]]


1/1 [==============================] - 0s 265ms/step


1/1 [==============================] - 1s 512ms/step - loss: 0.0000e+00 - accuracy: 0.9350 - binary_crossentropy: 0.2474


### check the model

In [6]:
for _model in models:
    inspector = models[_model].make_inspector()
    print(_model, inspector.evaluation())

fold_1 Evaluation(num_examples=493, accuracy=0.8630130665280685, loss=0.39728187565696926, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_2 Evaluation(num_examples=493, accuracy=0.8879849302451578, loss=0.35802094862341904, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_3 Evaluation(num_examples=494, accuracy=0.8662722815245514, loss=0.3867007871614992, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_4 Evaluation(num_examples=494, accuracy=0.8484623195658862, loss=0.387730822334424, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)
fold_5 Evaluation(num_examples=494, accuracy=0.8626250812558256, loss=0.38780961618679927, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)


# Prediction

## Preparing test data

In [7]:
test_ds_pd = test_df
test_df_columns = test_ds_pd.columns.tolist()
TEST_FEATURE_COLUMNS = [i for i in FEATURE_COLUMNS \
                        if i in test_df_columns and i != "Class"]
test_ds_pd = test_ds_pd[TEST_FEATURE_COLUMNS]
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd)

In [8]:
## Making Prediction

In [9]:
# Ensemble Predictions
predictions = np.zeros((len(test_df), len(models)))
for i, model in enumerate(models.values()):
    predictions[:, i] = model.predict(test_ds).ravel()

# Averaging the predictions
average_predictions = predictions.mean(axis=1)
n_predictions = [[round(abs(i-1), 8), i] for i in average_predictions]
print(n_predictions)

1/1 [==============================] - 0s 212ms/step


2023-06-30 19:43:40.851009: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype double and shape [5]
	 [[{{node Placeholder/_6}}]]


1/1 [==============================] - 0s 220ms/step
[[0.6743997, 0.32560030221939085], [0.6743997, 0.32560030221939085], [0.6743997, 0.32560030221939085], [0.6743997, 0.32560030221939085], [0.6743997, 0.32560030221939085]]


# Submission

In [10]:
sample_submission[['class_0', 'class_1']] = n_predictions
sample_submission.to_csv('submission.csv', index=False)